In [1]:
!pip install rdkit
!pip install SmilesPE

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 8.5 MB/s eta 0:00:00


In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import Descriptors
from rdkit.Chem import Descriptors3D
from rdkit.Chem import rdMolDescriptors
import seaborn as sns

In [5]:
from SmilesPE.pretokenizer import atomwise_tokenizer
from SmilesPE.tokenizer import *
import codecs

In [6]:
from joblib import dump, load
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from catboost import CatBoostRegressor
import math

In [8]:
df1 = pd.read_csv('/content/drive/MyDrive/NTO_FINAL/Информация о связывании медицинского радионуклида различными молекулами.csv')

In [9]:
df1 = df1.sort_values(by='lgK', ascending=False).iloc[:105]

In [10]:
df2 = pd.read_csv('/content/drive/MyDrive/NTO_FINAL/submits/gen_top_valid_first.csv', names=['smiles'])

In [11]:
df3 = pd.read_csv('/content/drive/MyDrive/NTO_FINAL/submits/gen_third_submit.csv', names=['smiles'])

In [12]:
df = pd.concat([df1, df2, df3], axis=0)

In [13]:
s = list(df['smiles'])

In [15]:
tokens = []

In [16]:
for smile in s:
  t = atomwise_tokenizer(smile)
  for i in t:
    if i not in tokens:
      tokens.append(i)

In [17]:
spe_vob = codecs.open('/content/drive/MyDrive/NTO_FINAL/SPE_ChEMBL.txt')
spe = SPE_Tokenizer(spe_vob)

In [18]:
for smile in s:
  t = spe.tokenize(smile).split(' ')
  for i in t:
    if i not in tokens:
      tokens.append(i)

In [19]:
len(tokens)

254

In [20]:
from itertools import combinations
from IPython.display import clear_output

In [ ]:
generations = []

In [ ]:
comb = list(combinations(tokens, 2))
for c in comb:
  r = ''.join([''.join(i) for i in c])
  m = Chem.MolFromSmiles(r,sanitize=False)
  if m != None:
    generations.append(r)
  clear_output(wait=True)

[20:49:15] SMILES Parse Error: extra open parentheses for input: 'CN1CCN(CNCC'


In [ ]:
len(generations)

In [ ]:
gen = pd.DataFrame(generations, columns=['smiles'])
gen.to_csv('meta1.csv')

In [49]:
gens = []

In [ ]:
for smile in s:
  t = spe.tokenize(smile).split(' ')
  for i in range(15, 21):
    comb = list(combinations(t, i))
    for c in comb:
      r = ''.join([''.join(i) for i in c])
      m = Chem.MolFromSmiles(r,sanitize=True)
      if m != None:
        gens.append(r)

In [57]:
len(gens)

54

In [58]:
temp2 = []
for i in gens:
  if i not in s:
    temp2.append(i)

In [59]:
temp2

['CN(CC(C)=O)Cc1ccn(-c2cccc(-n3ccc(CN(CC(C)=O)CC(C)=O)n3)n2)n1',
 'O=C(O)c1ccn(Cc2cc(ccc(C(=O)O)n3)c(ccc(C(=O)O)n3)cc2Cn2ccc(C(=O)O)n2)n1',
 'O=C(O)c1ccc2cc(Cn3ccc(C(=O)O)n3)c(Cn3ccc(C(=O)O)n3)cc2Cn2ccc(n2)n1',
 'O=C(O)c1ccc2cc(Cn3ccc(C(=O)O)n3)c(Cn3ccc(n3)cc2Cn2ccc(C(=O)O)n2)n1',
 'O=C(O)c1ccc2cc(Cn3ccc(n3)c(Cn3ccc(C(=O)O)n3)cc2Cn2ccc(C(=O)O)n2)n1',
 'c1ccn(Cc2cc(Cn3ccc(C(=O)O)n3)c(Cn3ccc(C(=O)O)n3)cc2Cn2ccc(C(=O)O)n2)n1',
 'O=C(O)C1=CC=CC=C(C(=O)O)C(Cn2ccc(C(=O)O)n2)CC2C(=N1)C=C2',
 'O=C(O)C1=CC=CC=C(C(=O)O)C(ccc(C(=O)O)n2)=CC2=NC(=N1)',
 'O=C(O)C1=CC=CC=C(C(Cn2ccc(C(=O)O)n2)=CC2=NN1)C=C2',
 'O=C(O)C1=CC=CCC(Cn2ccc(C(=O)O)n2)=CC2=NC(=N1)C=C2',
 'O=C(O)C1=CC=C(C(=O)O)C(Cn2ccc(C(=O)O)n2)=CC2C(=N1)C=C2',
 'O=C(O)C1=CC=C(C(=O)O)C(Cn2ccc(C(=O)O)n2)CC2=NC(=N1)C=C2',
 'C1=CC=CC=C(C(=O)O)C(Cn2ccc(C(=O)O)n2)=CC2C(=N1)C=C2',
 'C1=CC=CC=C(C(=O)O)C(Cn2ccc(C(=O)O)n2)CC2=NC(=N1)C=C2',
 'C1=CC=C(C(=O)O)C(Cn2ccc(C(=O)O)n2)=CC2=NC(=N1)C=C2',
 'O=C(O)C1=CC=CC=C(C(=O)O)C(Cn2ccc(C(=O)O)n2)=CC2C(=N1)C=C2

In [34]:
len(temp2)

2923

In [35]:
ge = pd.DataFrame(temp2, columns=['smiles'])
ge.to_csv('meta3.csv')